### Generation of unoptimized circuit

In [1]:
### Input information ###
num_samples = 1
nqubits = 5
with_swap_option = False # If True, the MCR swap is executed
# Number of iterations for the unoptimized circuit
unopt_iteration_count = nqubits ** 2

In [2]:
from mcr.rotation_circuit import RotationCircuit, rot_unoptimization
from mcr.circuit_ops import equivalence_check_via_mqt_qcec
import stim

circuit = RotationCircuit(nqubits)
initial_pauli_string = "Z" * nqubits
circuit.add_gate((0,), stim.PauliString(f"+{initial_pauli_string}"))
# circuit.add_gate((1,), stim.PauliString(f"-{initial_pauli_string}"))
circuit_initial = circuit.duplicate()

# Perform unoptimization
unopt_circuit = rot_unoptimization(circuit, unopt_iteration_count, with_swap_option)
u = circuit_initial.set_circuit()
v = unopt_circuit.set_circuit()
circuit_u_rots = circuit_initial.to_rot_ops()
circuit_v_rots = unopt_circuit.to_rot_ops()
if nqubits <= 6:
    assert unopt_circuit.is_equivalent(circuit_initial), "The circuit is not equivalent to the original one."

### Compiler evaluation

### Data analysis